In [179]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [180]:
import numpy as np
import pandas as pd
import nltk
import re
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from nltk import word_tokenize
from scipy.sparse import hstack
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [270]:
df_train = pd.read_csv("drive/My Drive/UST/6000H/ml4nlp-argimpact/train.csv")
df_test = pd.read_csv("drive/My Drive/UST/6000H/ml4nlp-argimpact/test.csv")
df_valid = pd.read_csv("drive/My Drive/UST/6000H/ml4nlp-argimpact/valid.csv")

In [271]:
df_train = df_train.append(df_valid)

In [272]:
class_names = df_train.impact_label.unique().tolist()

In [273]:
# df_train['prev_argument'] = df_train.apply(lambda row: eval(row.context)[-1], axis=1)
# df_test['prev_argument'] = df_test.apply(lambda row: eval(row.context)[-1], axis=1)
# df_valid['prev_argument'] = df_valid.apply(lambda row: eval(row.context)[-1], axis=1)

df_train['context'] = df_train.apply(lambda row: ' '.join(eval(row.context)), axis=1)
df_test['context'] = df_test.apply(lambda row: ' '.join(eval(row.context)), axis=1)
# df_valid['context'] = df_valid.apply(lambda row: ' '.join(eval(row.context)), axis=1)

df_test['text'] = df_test.apply(lambda row: eval(row.text)[0], axis=1)

In [274]:
# df_train['cleaned_text'] = df_train['text'].map(lambda x: cleanData(x))
# df_test['cleaned_text'] = df_test['text'].map(lambda x: cleanData(x))
# df_valid['cleaned_text'] = df_valid['text'].map(lambda x: cleanData(x))

# df_train['cleaned_context'] = df_train['context'].map(lambda x: cleanData(x))
# df_test['cleaned_context'] = df_test['context'].map(lambda x: cleanData(x))
# df_valid['cleaned_context'] = df_valid['context'].map(lambda x: cleanData(x))

# df_train['cleaned_prev_argument'] = df_train['prev_argument'].map(lambda x: cleanData(x))
# df_test['cleaned_prev_argument'] = df_test['prev_argument'].map(lambda x: cleanData(x))
# df_valid['cleaned_prev_argument'] = df_valid['prev_argument'].map(lambda x: cleanData(x))

In [275]:
# def train_word2vec(df):

#     texts = (df["cleaned_context"] + df["cleaned_text"]).to_numpy().tolist()

#     tok_texts = [word_tokenize(text) for text in texts]

#     return Word2Vec(tok_texts, sg=1, size=100, window=10, min_count=5, workers=4,
#                     iter=100)

In [276]:
# train_w2v_model = train_word2vec(df_train)
# test_w2v_model = train_word2vec(df_test)
# valid_w2v_model = train_word2vec(df_valid)

In [277]:
# train_w2v_model.save('drive/My Drive/UST/6000H/ml4nlp-argimpact/train_w2v.model')
# test_w2v_model.save('drive/My Drive/UST/6000H/ml4nlp-argimpact/test_w2v.model')
# valid_w2v_model.save('drive/My Drive/UST/6000H/ml4nlp-argimpact/valid_w2v.model')

In [278]:
# train_w2v_model = Sentence2Vec('drive/My Drive/UST/6000H/ml4nlp-argimpact/train_w2v.model')
# test_w2v_model = Sentence2Vec('drive/My Drive/UST/6000H/ml4nlp-argimpact/test_w2v.model')
# valid_w2v_model = Sentence2Vec('drive/My Drive/UST/6000H/ml4nlp-argimpact/valid_w2v.model')

In [279]:
# df_train["s2v_similarity"] = df_train.apply(lambda row: train_w2v_model.similarity(row.cleaned_text, row.cleaned_prev_argument), axis=1)

In [280]:
# df_train.groupby(by='impact_label').mean()

In [281]:
df_train['impact_label'] = df_train.impact_label.map({"NOT_IMPACTFUL": 0, "MEDIUM_IMPACT": 1, "IMPACTFUL": 2})
# df_valid['impact_label'] = df_valid.impact_label.map({"NOT_IMPACTFUL": 0, "MEDIUM_IMPACT": 1, "IMPACTFUL": 2})

In [282]:
df_train['cleaned_stance_label'] = df_train['stance_label'].apply(lambda x: ' '.join(eval(x)[1:]))
df_test['cleaned_stance_label'] = df_test['stance_label'].apply(lambda x: ' '.join(eval(x)[1:]))
# df_valid['cleaned_stance_label'] = df_valid['stance_label'].apply(lambda x: ' '.join(eval(x)[1:]))

In [283]:
# df_train['previous_argument_type'] = df_train['stance_label'].apply(lambda x: eval(x)[-1])
# df_test['previous_argument_type'] = df_test['stance_label'].apply(lambda x: eval(x)[-1])
# df_valid['previous_argument_type'] = df_valid['stance_label'].apply(lambda x: eval(x)[-1])

In [284]:
# df_train['previous_argument_type'] = df_train.previous_argument_type.map({"OPPOSE": 0, "SUPPORT": 1})
# df_test['previous_argument_type'] = df_test.previous_argument_type.map({"OPPOSE": 0, "SUPPORT": 1})
# df_valid['previous_argument_type'] = df_valid.previous_argument_type.map({"OPPOSE": 0, "SUPPORT": 1})

In [285]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

In [286]:
text_count_vect = CountVectorizer(lowercase=True, stop_words="english", ngram_range=(1, 4))
context_count_vect = CountVectorizer(lowercase=True, stop_words="english", ngram_range=(1, 4))

In [287]:
stance_count_vect = CountVectorizer(lowercase=True, ngram_range=(1, 5))

In [288]:
X_train = count_vect.fit_transform(df_train['text'].to_numpy())
# X_valid = count_vect.transform(df_valid['text'].to_numpy())
X_test = count_vect.transform(df_test['text'].to_numpy())

In [289]:
train_context_count_vect = count_vect.fit_transform(df_train['context'].to_numpy())
# valid_context_count_vect = count_vect.transform(df_valid['context'].to_numpy())
test_context_count_vect = count_vect.transform(df_test['context'].to_numpy())

In [290]:
train_stance = stance_count_vect.fit_transform(df_train['cleaned_stance_label'].to_numpy())
# valid_stance = stance_count_vect.transform(df_valid['cleaned_stance_label'].to_numpy())
test_stance = stance_count_vect.transform(df_test['cleaned_stance_label'].to_numpy())

In [291]:
X_train = hstack((X_train, train_context_count_vect))
# X_valid = hstack((X_valid, valid_context_count_vect))
X_test = hstack((X_test, test_context_count_vect))
X_train = hstack((X_train, train_stance))
# X_valid = hstack((X_valid, valid_stance))
X_test = hstack((X_test, test_stance))

In [292]:
y_train = df_train['impact_label'].to_numpy()
# y_valid = df_valid['impact_label'].to_numpy()

In [293]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [294]:
# Number of trees in random forest
n_estimators = [100, 200, 500]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [5, 8, 10, 20, 30]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [295]:
sgd = SGDClassifier()
rfc = RandomForestClassifier()

In [296]:
# gs = GridSearchCV(rfc, param_grid=param_grid, verbose=1)

In [298]:
sgd.fit(X_train, y_train)
# rfc.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [269]:
validation_pred = sgd.predict(X_valid)
print(f"SGD F-1 score: {metrics.f1_score(df_valid.impact_label.to_numpy(), validation_pred, average='macro')}")

validation_pred = rfc.predict(X_valid)
print(f"RFC F-1 score: {metrics.f1_score(df_valid.impact_label.to_numpy(), validation_pred, average='macro')}")

SGD F-1 score: 0.5452076017456213
RFC F-1 score: 0.5089825751531633


In [ ]:
# text_clf.fit(df_train.text.to_numpy(), df_train.impact_label.to_numpy())

In [ ]:
# validation_pred = text_clf.predict(df_valid.text.to_numpy())

In [ ]:
# print(f"F-1 score: {metrics.f1_score(df_valid.impact_label.to_numpy(), validation_pred, average='macro')}")

In [ ]:
# metrics.classification_report(df_valid.impact_label.to_numpy(), validation_pred, target_names=class_names)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
    'feats__vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__alpha': (0.02, 0.03, 0.2),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, verbose=1)

In [ ]:
gs_clf.fit(df_train.text.to_numpy(), df_train.impact_label.to_numpy())

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   19.7s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('feats',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('vect',
                                                                        CountVectorizer(analyzer='word',
                                                                                        binary=False,
                                                                                        decode_error='strict',
                                                                                        dtype=<class 'numpy.int64'>,
                                                                                        encoding='utf-8',
                                                                                        input='content',
                                                                              

In [ ]:
gs_clf.cv_results_

{'mean_fit_time': array([0.33352962, 1.07944336, 2.03904166, 0.29629989, 1.03692803,
        1.89910464]),
 'mean_score_time': array([0.07668324, 0.15674677, 0.21972637, 0.06506205, 0.14441047,
        0.1965364 ]),
 'mean_test_score': array([0.61257253, 0.61121857, 0.60754352, 0.54158607, 0.5655706 ,
        0.59748549]),
 'param_clf__alpha': masked_array(data=[0.01, 0.01, 0.01, 0.001, 0.001, 0.001],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_feats__vect__ngram_range': masked_array(data=[(1, 1), (1, 2), (1, 3), (1, 1), (1, 2), (1, 3)],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 1)},
  {'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 2)},
  {'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 3)},
  {'clf__alpha': 0.001, 'feats__vect__ngram_range': (1, 1)},
  {'clf__al

In [ ]:
validation_pred = gs_clf.predict(df_valid.text.to_numpy())

In [ ]:
print(f"F-1 score: {metrics.f1_score(df_valid.impact_label.to_numpy(), validation_pred, average='macro')}")

F-1 score: 0.4852736607231976


In [ ]:
gs_clf.fit(df_train.text.to_numpy(), df_train.impact_label.to_numpy())

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   24.3s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('feats',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('vect',
                                                                        CountVectorizer(analyzer='word',
                                                                                        binary=False,
                                                                                        decode_error='strict',
                                                                                        dtype=<class 'numpy.int64'>,
                                                                                        encoding='utf-8',
                                                                                        input='content',
                                                                              

In [ ]:
gs_clf.cv_results_

{'mean_fit_time': array([0.40860076, 1.26529689, 2.40461206, 0.36888561, 1.25734696,
        2.24724402]),
 'mean_score_time': array([0.09286661, 0.17571011, 0.27506499, 0.08236485, 0.17906299,
        0.25027771]),
 'mean_test_score': array([0.6153257 , 0.62280889, 0.62185424, 0.55033433, 0.60178851,
        0.60895539]),
 'param_clf__alpha': masked_array(data=[0.01, 0.01, 0.01, 0.001, 0.001, 0.001],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_feats__vect__ngram_range': masked_array(data=[(1, 1), (1, 2), (1, 3), (1, 1), (1, 2), (1, 3)],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 1)},
  {'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 2)},
  {'clf__alpha': 0.01, 'feats__vect__ngram_range': (1, 3)},
  {'clf__alpha': 0.001, 'feats__vect__ngram_range': (1, 1)},
  {'clf__al

In [299]:
predictions = sgd.predict(X_test)

In [300]:
submission = df_test[['id']]

In [301]:
submission['pred'] = predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [302]:
submission.to_csv('drive/My Drive/UST/6000H/sgd.csv', index=False)